In [2]:
import numpy as np
import torch

Pieces:

WP1:8
WR1:2
WK1:2
WB1:2
WQ
WK

BP1:8
BR1:2
BK1:2
BB1:2
BQ
BK

In [73]:
WP1 = torch.randn(1000)
WP2 = torch.randn(1000)
WP3 = torch.randn(1000)
WP4 = torch.randn(1000)
WP5 = torch.randn(1000)
WP6 = torch.randn(1000)
WP7 = torch.randn(1000)
WP8 = torch.randn(1000)

WR1 = torch.randn(1000)
WR2 = torch.randn(1000)

WN1 = torch.randn(1000)
WN2 = torch.randn(1000)

WB1 = torch.randn(1000)
WB2 = torch.randn(1000) 

WQ = torch.randn(1000)
WK = torch.randn(1000)

BP1 = torch.randn(1000)
BP2 = torch.randn(1000)
BP3 = torch.randn(1000)
BP4 = torch.randn(1000)
BP5 = torch.randn(1000)
BP6 = torch.randn(1000)
BP7 = torch.randn(1000)
BP8 = torch.randn(1000)

BR1 = torch.randn(1000)
BR2 = torch.randn(1000)

BN1 = torch.randn(1000)
BN2 = torch.randn(1000)

BB1 = torch.randn(1000)
BB2 = torch.randn(1000)

BQ = torch.randn(1000)
BK = torch.randn(1000)   

E = torch.randn(1000)

In [74]:
import torch

# Define each row using lists of piece vectors (each of shape [1000])
row1 = [WR1, WN1, WB1, WQ, WK, WB2, WN2, WR2]
row2 = [WP1, WP2, WP3, WP4, WP5, WP6, WP7, WP8]
row3 = [E] * 8
row4 = [E] * 8
row5 = [E] * 8
row6 = [E] * 8
row7 = [BP1, BP2, BP3, BP4, BP5, BP6, BP7, BP8]
row8 = [BR1, BN1, BB1, BQ, BK, BB2, BN2, BR2]

# Stack all rows into one flat list, then stack into a 64x1000 tensor
board_rows = [row1, row2, row3, row4, row5, row6, row7, row8]
current_board = torch.stack([piece for row in board_rows for piece in row])  # shape: [64, 1000]


In [49]:
current_board.shape

torch.Size([64, 1000])

In [58]:
queryW = torch.randn([1000, 1000])
keyW = torch.randn([1000, 1000])
valueW = torch.randn([1000, 64])

In [60]:
query_embeddings = current_board @ queryW
key_embeddings = current_board @ keyW
value_embeddings = current_board @ valueW

In [62]:
final_emb = (query_embeddings @ key_embeddings.T) + value_embeddings
final_emb.shape

torch.Size([64, 64])

In [ ]:
max(torch.softmax(final_emb.reshape([64**2]), 0))

tensor(1.)

In [104]:
import chess
import torch

# ---------------------- 1. piece‑name ↔ vector lookup ----------------------
piece_vecs = {
    # white
    "WP1": WP1, "WP2": WP2, "WP3": WP3, "WP4": WP4,
    "WP5": WP5, "WP6": WP6, "WP7": WP7, "WP8": WP8,
    "WR1": WR1, "WR2": WR2,
    "WK1": WN1, "WK2": WN2,             # knights (you called them WK*)
    "WB1": WB1, "WB2": WB2,
    "WQ" : WQ , "WK" : WK,              # king
    # black
    "BP1": BP1, "BP2": BP2, "BP3": BP3, "BP4": BP4,
    "BP5": BP5, "BP6": BP6, "BP7": BP7, "BP8": BP8,
    "BR1": BR1, "BR2": BR2,
    "BK1": BN1, "BK2": BN2,             # knights (you called them BK*)
    "BB1": BB1, "BB2": BB2,
    "BQ" : BQ , "BK" : BK,              # king
}
EMPTY = E                              # 1 000‑d “empty‑square” vector

# Initial piece‑ID placement (square numbers are 0=a1 … 63=h8, python‑chess style)
initial_map = {
    # white back rank
    0: "WR1", 1: "WN1", 2: "WB1", 3: "WQ", 4: "WK", 5: "WB2", 6: "WN2", 7: "WR2",
    # white pawns
    8: "WP1", 9: "WP2", 10: "WP3", 11: "WP4",
    12: "WP5", 13: "WP6", 14: "WP7", 15: "WP8",
    # black pawns
    48: "BP1", 49: "BP2", 50: "BP3", 51: "BP4",
    52: "BP5", 53: "BP6", 54: "BP7", 55: "BP8",
    # black back rank
    56: "BR1", 57: "BN1", 58: "BB1", 59: "BQ",
    60: "BK",  61: "BB2", 62: "BN2", 63: "BR2",
}

# ---------------------- 2. helper: mapping ➜ 64×1000 tensor ---------------
def mapping_to_tensor(square2id: dict) -> torch.Tensor:
    """
    square2id : {square_idx: piece_id_name}
    returns   : torch.Tensor[64, 1000]
    """
    vecs = [piece_vecs.get(square2id.get(sq, None), EMPTY) for sq in range(64)]
    return torch.stack(vecs)     # [64, 1000]

# ---------------------- 4. aggregate many games ---------------------------
def games_to_tensor_dataset(game_dicts: list[dict]) -> torch.Tensor:
    """
    Flattens every board‑state from every game into one big tensor:
        shape = [num_positions , 64 , 1000]
    """
    all_states = []
    for g in game_dicts:
        all_states.extend(game_to_board_tensors(g))
    return torch.stack(all_states)

In [105]:
import datetime

games = [{'Event': 'Rated Classical game',
 'Site': 'https://lichess.org/j1dkb5dw',
 'White': 'BFG9k',
 'Black': 'mamalak',
 'Result': '1-0',
 'WhiteTitle': None,
 'BlackTitle': None,
 'WhiteElo': 1639,
 'BlackElo': 1403,
 'WhiteRatingDiff': 5,
 'BlackRatingDiff': -8,
 'UTCDate': datetime.date(2012, 12, 31),
 'UTCTime': datetime.time(23, 1, 3),
 'ECO': 'C00',
 'Opening': 'French Defense: Normal Variation',
 'Termination': 'Normal',
 'TimeControl': '600+8',
 'movetext': '1. e4 e6 2. d4 b6 3. a3 Bb7 4. Nc3 Nh6 5. Bxh6 gxh6 6. Be2 Qg5 7. Bg4 h5 8. Nf3 Qg6 9. Nh4 Qg5 10. Bxh5 Qxh4 11. Qf3 Kd8 12. Qxf7 Nc6 13. Qe8# 1-0'}]

In [110]:
import re
import chess
import torch

# ------------------------------------------------------------------ #
# 1.  TOKEN CLEAN‑UP  – turn raw movetext into a pure SAN generator  #
# ------------------------------------------------------------------ #
_move_num = re.compile(r"^\d+\.(\.\.)?$")   # matches “1.” or “1...”

def san_stream(movetext: str):
    """
    Yields pure SAN moves from a movetext string that may contain:
      * move numbers  (1.  5...  12.)
      * game results  (1-0  0-1  1/2-1/2  *)
    """
    for tok in movetext.replace("\n", " ").split():
        if _move_num.match(tok) or tok in {"1-0", "0-1", "1/2-1/2", "*"}:
            continue
        yield tok

# ------------------------------------------------------------------ #
# 2.  game_to_board_tensors – same logic, but call the new san_stream #
# ------------------------------------------------------------------ #
def game_to_board_tensors(game_dict) -> list[torch.Tensor]:
    board   = chess.Board()
    mapping = initial_map.copy()
    tensors = [mapping_to_tensor(mapping)]

    for san in san_stream(game_dict["movetext"]):
        move = board.parse_san(san)

        from_sq, to_sq = move.from_square, move.to_square
        moving_id = mapping.pop(from_sq)           # leave origin

        # capture on target square (regular or promotion capture)
        if to_sq in mapping:
            mapping.pop(to_sq)

        # ---------- en‑passant (ask the board, not the move) ----------
        if board.is_en_passant(move):
            ep_target = to_sq + (-8 if board.turn else 8)
            mapping.pop(ep_target, None)

        mapping[to_sq] = moving_id                 # land piece

        # ---------- castling (again, ask the board) -------|---
        if board.is_castling(move):
            if chess.square_file(to_sq) == chess.GFILE:   # king‑side 0‑0
                rook_from, rook_to = (chess.H1, chess.F1) if board.turn else (chess.H8, chess.F8)
            else:                                         # queen‑side 0‑0‑0
                rook_from, rook_to = (chess.A1, chess.D1) if board.turn else (chess.A8, chess.D8)
            mapping[rook_to] = mapping.pop(rook_from)

        board.push(move)    
        print(board)
        print(' ' * 20)
        tensors.append(mapping_to_tensor(mapping))

    return tensors

In [111]:
board_tensors = games_to_tensor_dataset(games)
print(board_tensors.shape)   # (total_positions, 64, 1000)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
                    
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
                    
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
                    
r n b q k b n r
p . p p . p p p
. p . . p . . .
. . . . . . . .
. . . P P . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
                    
r n b q k b n r
p . p p . p p p
. p . . p . . .
. . . . . . . .
. . . P P . . .
P . . . . . . .
. P P . . P P P
R N B Q K B N R
                    
r n . q k b n r
p b p p . p p p
. p . . p . . .
. . . . . . . .
. . . P P . . .
P . . . . . . .
. P P . . P P P
R N B Q K B N R
                    
r n . q k b n r
p b p p . p p p
. p . . p . . .
. . . . . . . .
. . . P P . . .
P . N . . . . .
. P P . . 